In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [19]:
import kagglehub

path = kagglehub.dataset_download("CooperUnion/anime-recommendations-database")
print("Path to dataset files:", path)


file_path = f"{path}/anime.csv"
df_anime = pd.read_csv(file_path)

df_anime.head()

Path to dataset files: C:\Users\ASUS\.cache\kagglehub\datasets\CooperUnion\anime-recommendations-database\versions\1


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


menghapus nilai nan

In [20]:
df_anime['genre'] = df_anime['genre'].fillna('')


Encode di kolom genre

In [21]:
model = SentenceTransformer('all-MiniLM-L6-v2')
genre_embeddings = model.encode(df_anime['genre'].tolist(), convert_to_tensor=True)


In [22]:
similarity_matrix = cosine_similarity(genre_embeddings.cpu().numpy())

In [23]:
def recommend_anime(anime_name, df_anime, similarity_matrix, top_n=5):
    index = df_anime[df_anime['name'] == anime_name].index
    if len(index) == 0:
        return f"Anime '{anime_name}' tidak ditemukan."
    
    index = index[0]

    sim_scores = list(enumerate(similarity_matrix[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1] 
    
    return df_anime.iloc[[i[0] for i in sim_scores]][['name', 'genre']]


In [24]:
recommend_anime("Kimi no Na wa.", df_anime, similarity_matrix)


,name,genre
5805,Wind: A Breath of Heart OVA,"Drama, Romance, School, Supernatural"
6394,Wind: A Breath of Heart (TV),"Drama, Romance, School, Supernatural"
1111,Aura: Maryuuin Kouga Saigo no Tatakai,"Comedy, Drama, Romance, School, Supernatural"
1959,Air Movie,"Drama, Romance, Supernatural"
4514,Touka Gettan,"Drama, Romance, Supernatural"
